In [1]:
server_args = []
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

from manipulation import running_as_notebook

# Imports
import numpy as np
import pydot
from ipywidgets import Dropdown, Layout
from IPython.display import display, HTML, SVG

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge,  AutoDiffXd, autoDiffToGradientMatrix, SnoptSolver, IpoptSolver,
    initializeAutoDiffGivenGradientMatrix, autoDiffToValueMatrix, autoDiffToGradientMatrix,
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint,
    MeshcatContactVisualizer, Expression
)

import pickle
from pydrake.symbolic import Variable

In [2]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)

In [3]:
#add table
ground = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/ground.urdf"), "ground")

#add box
cracker_box = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/003_cracker_box_plannar_test.urdf"), "cracker_box")

X_WB = RigidTransform(RotationMatrix.MakeXRotation(np.pi/2), [0., 0., 0.104200 + 1e-07])#
planar_joint_frame = plant.AddFrame(pydrake.multibody.tree.FixedOffsetFrame("planar_joint_frame", plant.world_frame(), X_WB))
box_joint = plant.AddJoint(pydrake.multibody.tree.PlanarJoint("box_joint", planar_joint_frame, plant.GetFrameByName("base_link_cracker", cracker_box)))
box_joint.set_default_translation([0, 0])

In [4]:
plant.Finalize()
plant.set_name("plant")

In [5]:
cracker_box_body = plant.GetBodyByName("base_link_cracker", cracker_box)
cracker_box_id = plant.GetBodyFrameIdIfExists(cracker_box_body.index())


draw_frames = True
frames_to_draw = [cracker_box_id] if draw_frames else []
# meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, frames_to_draw=frames_to_draw, axis_length=0.15,
#                  axis_radius=0.001,)
# # meshcat.set_planar_viewpoint(xmin=-.5, xmax=.5, ymin=-0.1, ymax=0.6)

# #visualizer the contact
# contact_visualizer = builder.AddSystem(MeshcatContactVisualizer(meshcat, 
#                                        force_threshold=0.0,
#                                        contact_force_scale=0.05, 
#                                        contact_force_radius=0.001,   
#                                        plant=plant))
# builder.Connect(plant.get_contact_results_output_port(),
#                 contact_visualizer.GetInputPort("contact_results"))


diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
# X_WB = RigidTransform(RotationMatrix(), [0., 0., 0.0334 + 1e-07])
# plant.SetFreeBodyPose(plant_context, 
#                               plant.GetBodyByName("base_link_cracker", cracker_box),
#                               X_WB)

# meshcat.load()
diagram.Publish(context)

In [6]:
# query_object = plant.get_geometry_query_input_port().Eval(plant_context)
# scene_graph_inspector = query_object.inspector()
# collision_candidate_pairs = scene_graph_inspector.GetCollisionCandidates()
# signed_distance_pairs = query_object.ComputeSignedDistancePairwiseClosestPoints()
# for signed_distance_pair in signed_distance_pairs:
#     print("id_A: ", signed_distance_pair.id_A, "   id_B: ", signed_distance_pair.id_B)
#     print("p_ACa: ", signed_distance_pair.p_ACa, "   p_BCb: ", signed_distance_pair.p_BCb)
#     print("nhat_BA_W: ", signed_distance_pair.nhat_BA_W, "   distance: ", signed_distance_pair.distance)
#     print("########################")

In [7]:
init_from_file = False
# init_from_file = True
# tmpfolder = 'tmp/'
# with open(tmpfolder +  'test_only_box_passive_scenegraph_sliding_friction/sol.pkl', 'rb' ) as file:
#     q_o_ini, v_o_ini, vdot_o_ini, f_list_ini, alpha_beta_list_ini, f_static_list_ini, f_sliding_list_ini, sliding_c_list_ini = pickle.load( file )

In [8]:
# Need this because a==b returns True even if a = AutoDiffXd(1, [1, 2]), b= AutoDiffXd(2, [3, 4])
# That's the behavior of AutoDiffXd in C++, also.
def autoDiffArrayEqual(a,b):
    return np.array_equal(a, b) and np.array_equal(autoDiffToGradientMatrix(a), autoDiffToGradientMatrix(b))


N = 10


# Create one context per timestep (to maximize cache hits)
#to use scenegraph, so the plant context need derive from diagram
context_list = [diagram.CreateDefaultContext() for i in range(N)]
plant_context_list = [plant.GetMyContextFromRoot(context_list[i]) for i in range(N)]
# We could get rid of this by implementing a few more Jacobians in MultibodyPlant:
ad_diagram = diagram.ToAutoDiffXd()
ad_plant = ad_diagram.GetSubsystemByName("plant")

# Make a new autodiff context (to maximize cache hits)
#to use scenegraph, so the plant context need derive from diagram
ad_context_list = [ad_diagram.CreateDefaultContext() for i in range(N)]
ad_plant_context_list = [ad_plant.GetMyContextFromRoot(ad_context_list[i]) for i in range(N)]


total_mass = sum(plant.get_body(index).get_mass(context) for index in plant.GetBodyIndices(cracker_box))
gravity = plant.gravity_field().gravity_vector()



#time step constrain
T = 0.01
prog = MathematicalProgram()
# Time steps    
h = prog.NewContinuousVariables(N-1, "h")
prog.AddBoundingBoxConstraint([0.5*T/(N-1)]*(N-1), [2.0*T/(N-1)]*(N-1), h) 
prog.AddLinearConstraint(sum(h) >= .9*T)
prog.AddLinearConstraint(sum(h) <= 1.1*T)
prog.SetInitialGuess(h, [T/(N-1)]*(N-1))

#q, v, vdot constrain
nq_o = plant.num_positions(cracker_box)
nv_o = plant.num_velocities(cracker_box)
q_o = prog.NewContinuousVariables(nq_o, N, "q_o")
v_o = prog.NewContinuousVariables(nv_o, N, "v_o")
vdot_o = prog.NewContinuousVariables(nv_o, N-1, "vdot_o")

#initial constrain
q0_o = plant.GetPositions(plant_context, cracker_box)
v0_o = plant.GetVelocities(plant_context, cracker_box)
vdot0_o = v0_o
v0_o[0] = 0.5
# vdot0_o[0] = -9.81
prog.AddBoundingBoxConstraint(q0_o, q0_o, q_o[:,0])
prog.AddBoundingBoxConstraint(v0_o, v0_o, v_o[:,0])
for n in range(N-1):
#     prog.AddQuadraticErrorCost(np.diag([1]*nq_o), q0_o, q_o[:,n])
    prog.AddQuadraticErrorCost(np.diag([100]*nv_o), [0]*nv_o, v_o[:,n])
    if not init_from_file:
        prog.SetInitialGuess(q_o[:,n], q0_o)
#         if n < 10:
#             prog.SetInitialGuess(vdot_o[:,n], vdot0_o)
#         if n < 50:
#             q0_o[0] =  n*0.001
#             prog.SetInitialGuess(q_o[:,n], q0_o)
#             v0_o[0] = 0.5 - n*0.01
#             prog.SetInitialGuess(v_o[:,n], v0_o)
#         else:
#             q0_o[0] =  50*0.001
#             prog.SetInitialGuess(q_o[:,n], q0_o)
#             v0_o[0] = 0
#             prog.SetInitialGuess(v_o[:,n], v0_o)
    # Unit quaternions
#     AddUnitQuaternionConstraintOnPlant(plant, q_o[:,n], prog)

    
if init_from_file:
    prog.SetInitialGuess(q_o, q_o_ini)
    prog.SetInitialGuess(v_o, v_o_ini)
    prog.SetInitialGuess(vdot_o, vdot_o_ini)

In [9]:
# def velocity_position_constraint(vars, context_index):
#     h, q, v, qn = np.split(vars, [1, 1+nq_o, 1+nq_o+nv_o])
#     if isinstance(vars[0], AutoDiffXd):
#         if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_plant_context_list[context_index], cracker_box)):
#             ad_plant.SetPositions(ad_plant_context_list[context_index], q)
#         v_from_qdot = ad_plant.MapQDotToVelocity(ad_plant_context_list[context_index], (qn - q)/h)
#     else:
#         if not np.array_equal(q, plant.GetPositions(plant_context_list[context_index])):
#             plant.SetPositions(plant_context_list[context_index], q)
#         v_from_qdot = plant.MapQDotToVelocity(plant_context_list[context_index], (qn - q)/h)
#     return v - v_from_qdot
# for n in range(N-1):
#     prog.AddConstraint(
#         partial(velocity_position_constraint, context_index=n), 
#         lb=[0]*nv_o, ub=[0]*nv_o, 
#         vars=np.concatenate(([h[n]], q_o[:,n], v_o[:,n+1], q_o[:,n+1])))  #(implicit Euler)
    
#accelaration and velocity relation constrain(implicit Euler)
for n in range(N-1):
    q_v_constrain = prog.AddConstraint(eq(q_o[:, n+1], q_o[:,n] + h[n]*v_o[:,n+1]))#(implicit Euler)
    v_vdot_constrain = prog.AddConstraint(eq(v_o[:, n+1], v_o[:,n] + h[n]*vdot_o[:,n]))#(implicit Euler)
    q_v_constrain.evaluator().set_description(f"q_v_constrain_{n}")
    v_vdot_constrain.evaluator().set_description(f"v_vdot_constrain_{n}")

In [10]:
query_object = plant.get_geometry_query_input_port().Eval(plant_context)
scene_graph_inspector = query_object.inspector()
collision_candidate_pairs = scene_graph_inspector.GetCollisionCandidates()

#set the ignored_collision_pairs
cracker_box_body = plant.GetBodyByName("base_link_cracker", cracker_box)
cracker_box_id = plant.GetBodyFrameIdIfExists(cracker_box_body.index())
ground_body = plant.GetBodyByName("ground", ground)
ground_id = plant.GetBodyFrameIdIfExists(ground_body.index())
ground_geometry_id = scene_graph_inspector.GetGeometryIdByName(ground_id, pydrake.geometry.Role.kProximity, "ground::ground_collision")
box_collision_geometry_id = scene_graph_inspector.GetGeometryIdByName(cracker_box_id, pydrake.geometry.Role.kProximity, "cracker_box::box_collision")
point_collision1_geometry_id = scene_graph_inspector.GetGeometryIdByName(cracker_box_id, pydrake.geometry.Role.kProximity, "cracker_box::point_collision1")
point_collision3_geometry_id = scene_graph_inspector.GetGeometryIdByName(cracker_box_id, pydrake.geometry.Role.kProximity, "cracker_box::point_collision3")

ignored_collision_pairs = set([(ground_geometry_id, box_collision_geometry_id)
#                                ,(ground_geometry_id, point_collision1_geometry_id)
#                                ,(ground_geometry_id, point_collision3_geometry_id)

                              ])
#it's the top four collision point
collision_pairs1 = set([
                               (ground_geometry_id, point_collision1_geometry_id)
                               ,(ground_geometry_id, point_collision3_geometry_id)
                              ])



#declare the force and alpha beta decision variables, set collision_candidate_pair_list
collision_candidate_pair_list = []
f_list = []
alpha_beta_list = []

f_static_list = []
f_sliding_list = []
sliding_c_list = []
for n in range(N-1):
    f = []
    alpha_beta = []
    
    f_static = []
    f_sliding = []
    sliding_c = []
    for collision_candidate_pair in collision_candidate_pairs:
        if collision_candidate_pair not in ignored_collision_pairs:
            f.append(prog.NewContinuousVariables(3, 1, f"collision_point_{collision_candidate_pair[0]}_{collision_candidate_pair[1]}_{n}_contact_force"))
            alpha_beta.append(prog.NewContinuousVariables(2, 1, f"{collision_candidate_pair[0]}_{collision_candidate_pair[1]}_{n}_alpha_beta"))
            if n == 0 :
                collision_candidate_pair_list.append(collision_candidate_pair)
            
            f_static.append(prog.NewContinuousVariables(3, 1, f"collision_point_{collision_candidate_pair[0]}_{collision_candidate_pair[1]}_{n}_static_contact_force"))
            f_sliding.append(prog.NewContinuousVariables(3, 1, f"collision_point_{collision_candidate_pair[0]}_{collision_candidate_pair[1]}_{n}_sliding_contact_force"))
            sliding_c.append(prog.NewContinuousVariables(1, 1, f"{collision_candidate_pair[0]}_{collision_candidate_pair[1]}_{n}_sliding_c"))
            
    alpha_beta_list.append(alpha_beta)
    f_list.append(f)
    
    f_static_list.append(f_static)
    f_sliding_list.append(f_sliding)
    sliding_c_list.append(sliding_c)

N_f = len(collision_candidate_pair_list)

#set initial guess of f and alpha beta
if init_from_file:
    for n in range(N-1):
        for k in range(N_f):
            prog.SetInitialGuess(f_list[n][k], f_list_ini[n][k])
            prog.SetInitialGuess(alpha_beta_list[n][k], alpha_beta_list_ini[n][k])
            
            prog.SetInitialGuess(f_static_list[n][k], f_static_list_ini[n][k])
            prog.SetInitialGuess(f_sliding_list[n][k], f_sliding_list_ini[n][k])
            prog.SetInitialGuess(sliding_c_list[n][k], sliding_c_list_ini[n][k])
else:
    f0 = np.array([0, 0, (-gravity[2])*total_mass/2])
    alpha0 = (-gravity[2])*total_mass/2
    beta0 = 0
    alpha_beta0 = np.array([alpha0, beta0])
    
    f_sliding0 = np.array([-(-gravity[2])*total_mass/2, 0, (-gravity[2])*total_mass/2])
    sliding_c0 = np.array([2.0])
    for n in range(N-1):
        for k in range(N_f):
            if collision_candidate_pair_list[k] not in collision_pairs1:
                prog.SetInitialGuess(f_list[n][k], f0)
                prog.SetInitialGuess(alpha_beta_list[n][k], alpha_beta0)
        
#                 prog.SetInitialGuess(f_static_list[n][k], f0)
#                 prog.SetInitialGuess(f_sliding_list[n][k], f0)
#                 prog.SetInitialGuess(sliding_c_list[n][k], sliding_c0)
        
        

In [11]:
iter_count = 0
#reference drake sliding_friction_complementarity_constraint.cc file
def sliding_friction_complementarity_constraint(vars, context_index):
    q, v, alpha_beta, f, f_static, f_sliding, sliding_c = np.split(vars, 
                                        [nq_o, nq_o+nv_o, nq_o+nv_o+N_f*2, nq_o+nv_o+N_f*2+N_f*3, 
                                          nq_o+nv_o+N_f*2+N_f*3+N_f*3, nq_o+nv_o+N_f*2+N_f*3+N_f*3+N_f*3])
    #f and alpha_beta contain all collsion candidates force and corresponding alpha,beta
    #1 dimension vector reshape to needed dimension
    alpha_beta = np.array(alpha_beta).reshape(N_f, 2)
    f = np.array(f).reshape(N_f, 3)
    
    f_static = np.array(f_static).reshape(N_f, 3)
    f_sliding = np.array(f_sliding).reshape(N_f, 3)
    sliding_c = np.array(sliding_c).reshape(N_f, 1)

    
    #to deal the error,"PyFunctionConstraint: Output must be of scalar type float. Got AutoDiffXd instead."
    #need eval_plant and eval_context for both AutoDiffXd and scalar type input
    if isinstance(vars[0], AutoDiffXd):
        #set plant pos and vel
        if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_plant_context_list[context_index], cracker_box)):
            ad_plant.SetPositions(ad_plant_context_list[context_index], cracker_box, q)
        if not autoDiffArrayEqual(v, ad_plant.GetVelocities(ad_plant_context_list[context_index], cracker_box)):
            ad_plant.SetVelocities(ad_plant_context_list[context_index], cracker_box, v)
        
        eval_plant = ad_plant
        eval_plant_context = ad_plant_context_list[context_index]
    else:
        if not np.array_equal(q, plant.GetPositions(plant_context_list[context_index], cracker_box)):
            plant.SetPositions(plant_context_list[context_index], cracker_box, q)
        if not np.array_equal(v, plant.GetVelocities(plant_context_list[context_index], cracker_box)):
            plant.SetVelocities(plant_context_list[context_index], cracker_box, v)
        
        eval_plant = plant
        eval_plant_context = plant_context_list[context_index]
    
    #get query object and scenegraph inspector
    eval_query_object = eval_plant.get_geometry_query_input_port().Eval(eval_plant_context)
    eval_scene_graph_inspector = eval_query_object.inspector()
    
    global iter_count
    if iter_count % (100*N) == 0:
        print("iter_count:",iter_count/N)
    iter_count = iter_count + 1
    res_list = []
    #f is A exert to B express in W
    for n in range(N_f):
        signed_distance_pair = eval_query_object.ComputeSignedDistancePairClosestPoints(
                                    collision_candidate_pair_list[n][0], collision_candidate_pair_list[n][1])
        #derive friction coefficience
        geometryA_props = eval_scene_graph_inspector.GetProximityProperties(signed_distance_pair.id_A)
        geometryB_props = eval_scene_graph_inspector.GetProximityProperties(signed_distance_pair.id_B)
        geometryA_friction = geometryA_props.GetProperty("material", "coulomb_friction")
        geometryB_friction = geometryB_props.GetProperty("material", "coulomb_friction")
        combined_friction = pydrake.multibody.plant.CalcContactFrictionFromSurfaceProperties(
                                                        geometryA_friction, geometryB_friction)
        
        #calc the velocity of contact point Cb of geometry B  related to frame A express in world frame
        frame_A_id = eval_scene_graph_inspector.GetFrameId(signed_distance_pair.id_A)
        frame_B_id = eval_scene_graph_inspector.GetFrameId(signed_distance_pair.id_B)
        frameA = eval_plant.GetBodyFromFrameId(frame_A_id).body_frame()
        frameB = eval_plant.GetBodyFromFrameId(frame_B_id).body_frame()
        
        V_AB_W = frameB.CalcSpatialVelocity(eval_plant_context, frameA, eval_plant.world_frame())
        
        X_BGb = eval_scene_graph_inspector.GetPoseInFrame(signed_distance_pair.id_B)
        p_GbCb = signed_distance_pair.p_BCb
        if isinstance(vars[0], AutoDiffXd):
            p_BCb = (X_BGb.cast[AutoDiffXd]()).multiply(p_GbCb.reshape(3, 1))
        else:
            p_BCb = X_BGb.multiply(p_GbCb.reshape(3, 1))
        p_BCb_W = (eval_plant.CalcRelativeTransform(eval_plant_context, 
                                                 eval_plant.world_frame(), frameB).rotation()).multiply(p_BCb.reshape(3, 1))
        v_ACb_W = V_AB_W.Shift(p_BCb_W).translational()
        
        
        
        nhat_BA_W = np.array([signed_distance_pair.nhat_BA_W]).T
        
        v_sliding_ACb_W = (np.identity(3) - nhat_BA_W * nhat_BA_W.T).dot(v_ACb_W)
        f_sliding_tangential = (np.identity(3) - nhat_BA_W * nhat_BA_W.T).dot(f_sliding[n])
        cs1 = f[n] - f_static[n] - f_sliding[n]
        cs1 = np.array([cs1[0], cs1[2]])
        cs2 = (-nhat_BA_W.T.dot(f_static[n]))[0]*v_sliding_ACb_W
        cs2 = np.array([cs2[0], cs2[2]])
        cs3 = f_sliding_tangential + sliding_c[n][0] * v_sliding_ACb_W
        cs3 = np.array([cs3[0], cs3[2]])
        #sliding_friction_complementarity_constraint
        res1 = np.concatenate((
        cs1,
        cs2,
        (-nhat_BA_W.T.dot(f_sliding[n])),
        [f_sliding[n].dot(((combined_friction.dynamic_friction()**2 + 1) * nhat_BA_W*nhat_BA_W.T - np.identity(3)).dot(f_sliding[n]))],
#         (-nhat_BA_W.T.dot(f_static[n])),
        [f_static[n].dot(((combined_friction.static_friction()**2 + 1) * nhat_BA_W*nhat_BA_W.T - np.identity(3)).dot(f_static[n]))],
        cs3,
        sliding_c[n])) #11
        
               
        #static friction cone complementarity constrain
        res2 = np.concatenate((
            [alpha_beta[n][0]],
            [alpha_beta[n][1]],
#             [f[n].dot(((combined_friction.static_friction()**2 + 1) * nhat_BA_W*nhat_BA_W.T - np.identity(3)).dot(f[n]))],
            (-nhat_BA_W.T.dot(f[n])) - [alpha_beta[n][0]],  
            [signed_distance_pair.distance - alpha_beta[n][1]], 
            [alpha_beta[n][0] * alpha_beta[n][1]]))  #6
        
        res = np.concatenate((res1, res2))
        res_list.append(res)
    if iter_count % (100*N) == 0:
        print("res_list:", autoDiffToValueMatrix(res_list)
             )
    #return must be 1 dimension vector, like [r1, r2, ...]
    res_list = np.array(res_list).reshape((10+5)*N_f,)
    return res_list

epsilon1 = 1e-8
epsilon2 = 1e-8
for n in range(N-1):
    #set vars be 1 dimension vector, like [r1, r2, ...]
    vars = np.concatenate((q_o[:, n+1], v_o[:, n+1], 
                            np.array(alpha_beta_list[n]).flatten(), 
                            np.array(f_list[n]).flatten(),
                           
                            np.array(f_static_list[n]).flatten(),
                            np.array(f_sliding_list[n]).flatten(),
                            np.array(sliding_c_list[n]).flatten() 
                          ))
    sliding_friction_constraint = prog.AddConstraint(
        partial(sliding_friction_complementarity_constraint, context_index=n), 
        lb=np.array([np.concatenate(([0]*2, [-epsilon1]*2,     [0]*2,     [0],      [0]*2, [0],           [0]*5)) for i in range(N_f)]).reshape((10+5)*N_f,), 
        ub=np.array([np.concatenate(([0]*2, [epsilon1]*2, [np.inf], [0],  [np.inf],  [0]*2, [np.inf],      [4*(-gravity[2])*total_mass], [np.inf], [0]*2, [epsilon2])) for i in range(N_f)]).reshape((10+5)*N_f,), 
        vars=vars)   
    sliding_friction_constraint.evaluator().set_description(f"sliding_friction_complementarity_constraint_{n}")

In [12]:
iter_count1 = 0
def manipulator_equations(vars, context_index):
    q, v, vdot, f = np.split(vars, [nq_o, nq_o+nv_o, nq_o+nv_o*2])
    #f and alpha_beta contain all collsion candidates force and corresponding alpha,beta
    f = np.array(f).reshape(N_f, 3)
    
    #to deal the error,"PyFunctionConstraint: Output must be of scalar type float. Got AutoDiffXd instead."
    #need eval_plant and eval_context for both AutoDiffXd and scalar type input
    if isinstance(vars[0], AutoDiffXd):
        #set plant pos and vel
        if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_plant_context_list[context_index], cracker_box)):
            ad_plant.SetPositions(ad_plant_context_list[context_index], cracker_box, q)
        if not autoDiffArrayEqual(v, ad_plant.GetVelocities(ad_plant_context_list[context_index], cracker_box)):
            ad_plant.SetVelocities(ad_plant_context_list[context_index], cracker_box, v)
        
        eval_plant = ad_plant
        eval_plant_context = ad_plant_context_list[context_index]
    else:
        if not np.array_equal(q, plant.GetPositions(plant_context_list[context_index], cracker_box)):
            plant.SetPositions(plant_context_list[context_index], cracker_box, q)
        if not np.array_equal(v, plant.GetVelocities(plant_context_list[context_index], cracker_box)):
            plant.SetVelocities(plant_context_list[context_index], cracker_box, v)
        
        eval_plant = plant
        eval_plant_context = plant_context_list[context_index]
    
    
    # matrices for the manipulator equations
    M = eval_plant.CalcMassMatrixViaInverseDynamics(eval_plant_context)
    Cv = eval_plant.CalcBiasTerm(eval_plant_context)
    tauG = eval_plant.CalcGravityGeneralizedForces(eval_plant_context)

    
    #tauF = sumi(J.T*F), use object query and scenegraph inspector to calc J
    tauF = np.zeros((nv_o))
    #get query object and scenegraph inspector
    eval_query_object = eval_plant.get_geometry_query_input_port().Eval(eval_plant_context)
    eval_scene_graph_inspector = eval_query_object.inspector()
    
    global iter_count1
    iter_count1 = iter_count1 + 1
    #f is A exert to B express in W
    for n in range(N_f):
        signed_distance_pair = eval_query_object.ComputeSignedDistancePairClosestPoints(
                                    collision_candidate_pair_list[n][0], collision_candidate_pair_list[n][1])
        
        frame_A_id = eval_scene_graph_inspector.GetFrameId(signed_distance_pair.id_A)
        frame_B_id = eval_scene_graph_inspector.GetFrameId(signed_distance_pair.id_B)
        frameA = eval_plant.GetBodyFromFrameId(frame_A_id).body_frame()
        frameB = eval_plant.GetBodyFromFrameId(frame_B_id).body_frame()
        
        X_AGa = eval_scene_graph_inspector.GetPoseInFrame(signed_distance_pair.id_A)
        p_GaCa = signed_distance_pair.p_ACa
        if isinstance(vars[0], AutoDiffXd):
            p_ACa = (X_AGa.cast[AutoDiffXd]()).multiply(p_GaCa.reshape(3, 1))
        else:
            p_ACa = X_AGa.multiply(p_GaCa.reshape(3, 1))
            
        X_BGb = eval_scene_graph_inspector.GetPoseInFrame(signed_distance_pair.id_B)
        p_GbCb = signed_distance_pair.p_BCb
        if isinstance(vars[0], AutoDiffXd):
            p_BCb = (X_BGb.cast[AutoDiffXd]()).multiply(p_GbCb.reshape(3, 1))
        else:
            p_BCb = X_BGb.multiply(p_GbCb.reshape(3, 1))
        
        
        Jv_v_WCa = eval_plant.CalcJacobianTranslationalVelocity(eval_plant_context, JacobianWrtVariable.kV,
                                                    frameA, p_ACa,
                                                    eval_plant.world_frame(), eval_plant.world_frame())
        Jv_v_WCb = eval_plant.CalcJacobianTranslationalVelocity(eval_plant_context, JacobianWrtVariable.kV,
                                                    frameB, p_BCb,
                                                    eval_plant.world_frame(), eval_plant.world_frame())
        tauF = tauF + Jv_v_WCa.T.dot(-f[n]) +  Jv_v_WCb.T.dot(f[n])

    global iter_count
    if iter_count1 % (100*N) == 0:
        print("manipulator_equations:", autoDiffToValueMatrix(M.dot(vdot) + Cv - tauG - tauF)
           ,"     tauF:", autoDiffToValueMatrix(tauF)
             )
    #return must be 1 dimension vector, like [r1, r2, ...]
    return M.dot(vdot) + Cv - tauG - tauF


# manipulator equations for all t (implicit Euler)
for n in range(N-1):
    vars = np.concatenate((q_o[:, n+1], v_o[:, n+1], vdot_o[:, n], 
                           np.array(f_list[n]).flatten()))
    manipulator_equations_constraint = prog.AddConstraint(partial(manipulator_equations, context_index=n), lb=[0]*nv_o, ub=[0]*nv_o, vars=vars)
    manipulator_equations_constraint.evaluator().set_description(f"manipulator_equations_constraint_{n}")

In [13]:
snopt = SnoptSolver().solver_id()
prog.SetSolverOption(snopt, 'Iterations Limits', 1e5 if running_as_notebook else 1)
prog.SetSolverOption(snopt, 'Major Iterations Limit', 200 if running_as_notebook else 1)
prog.SetSolverOption(snopt, 'Major Feasibility Tolerance', 5e-6)
prog.SetSolverOption(snopt, 'Major Optimality Tolerance', 1e-4)
prog.SetSolverOption(snopt, 'Superbasics limit', 2000)
prog.SetSolverOption(snopt, 'Linesearch tolerance', 0.9)
prog.SetSolverOption(snopt, 'Print file', 'snopt.out')


# ipopt = IpoptSolver()
# result = ipopt.Solve(prog)

# sno = SnoptSolver()
# result = sno.Solve(prog)

result = Solve(prog)
print(result.get_solver_id().name())
if not result.is_success():
#     print(result.GetInfeasibleConstraintNames(prog))
    print("solve failed!")
else:
    print("solve success")
    

iter_count: 0.0
SNOPT/fortran
solve failed!


In [14]:
print(result.GetInfeasibleConstraintNames(prog))

[]


In [15]:
q_o_sol = result.GetSolution(q_o)
print("q_o_sol:", q_o_sol)

q_o_sol: [[ 0.00000000e+00  1.02118747e-03  1.99160834e-03  2.47051282e-03
   2.72297127e-03  2.99426445e-03  3.24820575e-03  3.48958854e-03
   3.73090696e-03  3.97215355e-03]
 [ 0.00000000e+00  7.11789032e-06  5.08908831e-05  6.22425501e-05
   6.50299083e-05  6.44770698e-05  6.06182189e-05  5.40081788e-05
   4.44338716e-05  3.19013500e-05]
 [ 0.00000000e+00 -8.94694489e-05 -2.44417215e-04 -3.22886969e-04
  -3.64624142e-04 -4.09780932e-04 -4.52085838e-04 -4.92405745e-04
  -5.32840664e-04 -5.73641282e-04]]


In [16]:
h_sol = result.GetSolution(h)
print("h:", h_sol)

h: [0.00222222 0.00222222 0.00110024 0.00058028 0.00062413 0.00058424
 0.00055556 0.00055556 0.00055556]


In [17]:
v_o_sol = result.GetSolution(v_o)
print("v_o_sol:", v_o_sol)

v_o_sol: [[ 0.5         0.45946897  0.43668939  0.43527233  0.43515612  0.43483026
   0.43462545  0.43448903  0.43437315  0.43426278]
 [ 0.          0.01872315  0.01969785  0.01031829  0.00474391 -0.00105369
  -0.006581   -0.01189807 -0.01723375 -0.02257732]
 [ 0.         -0.04032617 -0.06972649 -0.0713226  -0.07184112 -0.07220043
  -0.07243371 -0.07257583 -0.07278285 -0.07342347]]


In [18]:
vdot_o_sol = result.GetSolution(vdot_o)
print("vdot_o_sol:", vdot_o_sol)

vdot_o_sol: [[-18.23939355 -10.25081197  -1.28798238  -0.20011142  -0.52234639
   -0.35058749  -0.2455624   -0.20857411  -0.1986515 ]
 [  8.42564795   0.43861455  -8.52503688  -9.60619058  -9.28889039
   -9.46062645  -9.57073574  -9.6042252   -9.61845868]
 [-18.1471462  -13.2301443   -1.45068903  -0.89395148  -0.5754579
   -0.39927178  -0.25581727  -0.37263387  -1.15319999]]


In [19]:
collision_point_contact_force_sol = np.zeros((N_f, 3, N-1))
f_list_sol = []
for n in range(N-1):
    f_sol = []
    for k in range(N_f):
        collision_point_contact_force_sol[k][:, n] = result.GetSolution(f_list[n][k]).reshape(3,)
        f_sol.append(result.GetSolution(f_list[n][k]))
    f_list_sol.append(f_sol)

res_f = np.zeros((3, N-1))
for k in range(N_f):
    res_f = res_f + collision_point_contact_force_sol[k]
print("collision_point_contact_force:", res_f)


collision_point_contact_force: [[-1.82393935 -1.0250812  -0.12879824 -0.02001114 -0.05223464 -0.03505875
  -0.02455624 -0.02085741 -0.01986515]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 1.82356479  1.02486145  0.12849631  0.02038094  0.05211096  0.03493735
   0.02392643  0.02057748  0.01915413]]


In [20]:
collision_point_contact_force_f_static_sol = np.zeros((N_f, 3, N-1))
f_static_list_sol = []
for n in range(N-1):
    f_static_sol = []
    for k in range(N_f):
        collision_point_contact_force_f_static_sol[k][:, n] = result.GetSolution(f_static_list[n][k]).reshape(3,)
        f_static_sol.append(result.GetSolution(f_static_list[n][k]))
    f_static_list_sol.append(f_static_sol)

res_f_static = np.zeros((3, N-1))
for k in range(N_f):
    res_f_static = res_f_static + collision_point_contact_force_f_static_sol[k]
print("collision_point_contact_force_f_static:", res_f_static)

collision_point_contact_force_f_static: [[-2.19953910e-04  1.73589638e-05  5.32607127e-04  3.92113648e-04
  -2.78583235e-05  3.56158402e-04 -1.24772338e-04  1.31308235e-06
  -9.06967542e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 5.78170297e-08  7.07435915e-08  2.14739338e-08  7.28972497e-10
   6.66272741e-08 -1.59826360e-09 -3.05871553e-08  3.21372926e-08
   3.73591009e-08]]


In [21]:
collision_point_contact_force_f_sliding_sol = np.zeros((N_f, 3, N-1))
f_sliding_list_sol = []
for n in range(N-1):
    f_sliding_sol = []
    for k in range(N_f):
        collision_point_contact_force_f_sliding_sol[k][:, n] = result.GetSolution(f_sliding_list[n][k]).reshape(3,)
        f_sliding_sol.append(result.GetSolution(f_sliding_list[n][k]))
    f_sliding_list_sol.append(f_sliding_sol)

res_f_sliding = np.zeros((3, N-1))
for k in range(N_f):
    res_f_sliding = res_f_sliding + collision_point_contact_force_f_sliding_sol[k]
print("collision_point_contact_force_f_sliding:", res_f_sliding)

collision_point_contact_force_f_sliding: [[-1.8237194  -1.02509856 -0.12933085 -0.02040326 -0.05220678 -0.03541491
  -0.02443147 -0.02085872 -0.01977445]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 1.82356474  1.02486138  0.12849629  0.02038094  0.05211089  0.03493736
   0.02392646  0.02057745  0.01915409]]


In [22]:
collision_point_sliding_c_sol = np.zeros((N_f, 1, N-1))
sliding_c_list_sol = []
for n in range(N-1):
    sliding_c_sol = []
    for k in range(N_f):
        collision_point_sliding_c_sol[k][:, n] = result.GetSolution(sliding_c_list[n][k]).reshape(1,)
        sliding_c_sol.append(result.GetSolution(sliding_c_list[n][k]))
    sliding_c_list_sol.append(sliding_c_sol)

for k in range(N_f):
    print("collision_point_sliding_c_sol", k, ": ", collision_point_sliding_c_sol[k])

collision_point_sliding_c_sol 0 :  [[5.62338752e-05 0.00000000e+00 1.04735652e-03 3.56877344e-05
  2.01771551e-04 0.00000000e+00 8.93677848e-04 3.33088734e-04
  1.18093155e-03]]
collision_point_sliding_c_sol 1 :  [[1.74605850e-04 5.33996808e-04 8.33454443e-04 1.47850841e-05
  1.63591160e-05 3.05104289e-06 4.65036941e-05 3.05459804e-04
  2.20622875e-04]]
collision_point_sliding_c_sol 2 :  [[0.00012822 0.110478   0.00843141 0.0241922  0.00260763 0.00111261
  0.00089182 0.00603163 0.03545391]]
collision_point_sliding_c_sol 3 :  [[4.00539566 2.27607166 0.29189707 0.02346742 0.11934441 0.08180395
  0.05535211 0.04217978 0.00943934]]


In [23]:
collision_point_alpha_beta_sol = np.zeros((N_f, 2, N-1))
alpha_beta_list_sol = []
for n in range(N-1):
    alpha_beta_sol = []
    for k in range(N_f):
        collision_point_alpha_beta_sol[k][:, n] = result.GetSolution(alpha_beta_list[n][k]).reshape(2,)
        alpha_beta_sol.append(result.GetSolution(alpha_beta_list[n][k]))
    alpha_beta_list_sol.append(alpha_beta_sol)

for k in range(N_f):
    print("collision_point_alpha_beta_sol", k, ": ", collision_point_alpha_beta_sol[k])


collision_point_alpha_beta_sol 0 :  [[ 4.79846134e-08  4.79788130e-08  4.79766709e-08  6.02555058e-09
   4.19523342e-08  4.79786095e-08 -2.26216661e-08  2.64727741e-08
   4.01392465e-08]
 [ 2.08400005e-01  2.08431457e-01  2.08436568e-01  2.08436035e-01
   2.08431891e-01  2.08424667e-01  2.08414849e-01  2.08402058e-01
   2.08386280e-01]]
collision_point_alpha_beta_sol 1 :  [[2.04448522e-08 4.79685961e-08 1.38686376e-08 4.79636229e-08
  1.25926848e-08 4.79612255e-08 3.40945794e-08 4.19404657e-08
  4.79680970e-08]
 [2.08414230e-01 2.08470319e-01 2.08487907e-01 2.08494011e-01
  2.08497046e-01 2.08496548e-01 2.08493142e-01 2.08486780e-01
  2.08477489e-01]]
collision_point_alpha_beta_sol 2 :  [[ 1.04463930e-05  4.74421005e-02  3.60668751e-03  1.03442368e-02
   1.11330000e-03 -2.34143407e-08  2.93220725e-04  2.57424280e-03
   1.51251608e-02]
 [ 1.42311262e-05  7.03251584e-05  8.79174877e-05  9.40244473e-05
   9.70633984e-05  9.65696855e-05  9.31670608e-05  8.68094881e-05
   7.75229669e-05]]
c

In [24]:
for n in range(N-1):
    vars = np.concatenate((q_o_sol[:, n+1], v_o_sol[:, n+1], vdot_o_sol[:, n], 
                           np.array(f_list_sol[n]).flatten()
                          ))
    print("manipulator_equations:", autoDiffToValueMatrix(manipulator_equations(vars=vars, context_index=n)))

manipulator_equations: [[ 4.44089210e-16]
 [ 2.42250664e-13]
 [-1.06496617e-11]]
manipulator_equations: [[0.00000000e+00]
 [0.00000000e+00]
 [1.91574478e-09]]
manipulator_equations: [[5.55111512e-17]
 [1.38777878e-16]
 [6.07081940e-10]]
manipulator_equations: [[3.98986399e-16]
 [3.46944695e-18]
 [5.49776790e-10]]
manipulator_equations: [[ 0.00000000e+00]
 [-3.67761377e-16]
 [ 9.75878345e-11]]
manipulator_equations: [[ 1.38777878e-17]
 [-4.85722573e-16]
 [ 2.33300385e-10]]
manipulator_equations: [[-3.46944695e-18]
 [-2.15105711e-16]
 [ 3.40224666e-10]]
manipulator_equations: [[ 1.35308431e-16]
 [-1.07552856e-16]
 [ 5.35870898e-10]]
manipulator_equations: [[0.00000000e+00]
 [1.52655666e-16]
 [1.93838215e-09]]


In [25]:
# with open(tmpfolder + 'test_only_box_passive_scenegraph_sliding_friction/sol.pkl', 'wb') as file:
#     pickle.dump( [q_o_sol, v_o_sol, vdot_o_sol, f_list_sol, alpha_beta_list_sol, 
#                   f_static_list_sol, f_sliding_list_sol, sliding_c_list_sol], file )

In [26]:
print(gravity)

[ 0.    0.   -9.81]
